In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
def density_matrix(alpha):
    """Creates a density matrix from a pure state."""
    # DO NOT MODIFY anything in this code block
    psi = alpha * np.array([1, 0], dtype=float) + np.sqrt(1 - alpha**2) * np.array(
        [0, 1], dtype=float
    )
    psi = np.kron(psi, np.array([1, 0, 0, 0], dtype=float))
    return np.outer(psi, np.conj(psi))

In [3]:
dev = qml.device("default.mixed", wires=3)


@qml.qnode(dev)
def circuit(p, alpha, tampered_wire):
    """A quantum circuit that will be able to identify bitflip errors.

    DO NOT MODIFY any already-written lines in this function.

    Args:
        p (float): The bit flip probability
        alpha (float): The parameter used to calculate `density_matrix(alpha)`
        tampered_wire (int): The wire that may or may not be flipped (zero-index)

    Returns:
        Some expectation value, state, probs, ... you decide!
    """

    qml.QubitDensityMatrix(density_matrix(alpha), wires=[0, 1, 2])

    # QHACK #
    # Preprocessing
    others = [i for i in [0, 1, 2] if i != tampered_wire]
    qml.CNOT(wires=[tampered_wire, others[0]])
    qml.CNOT(wires=[tampered_wire, others[1]])
    # QHACK #

    qml.BitFlip(p, wires=int(tampered_wire))

    # QHACK #  
    # Post-processing
    qml.CNOT(wires=[tampered_wire, others[0]])
    qml.CNOT(wires=[tampered_wire, others[1]])
    qml.Toffoli(wires=[others[0], others[1], tampered_wire])
    return qml.state()

def error_wire(circuit_output):
    """Function that returns an error readout.

    Args:
        - circuit_output (?): the output of the `circuit` function.

    Returns:
        - (np.ndarray): a length-4 array that reveals the statistics of the
        error channel. It should display your algorithm's statistical prediction for
        whether an error occurred on wire `k` (k in {1,2,3}). The zeroth element represents
        the probability that a bitflip error does not occur.

        e.g., [0.28, 0.0, 0.72, 0.0] means a 28% chance no bitflip error occurs, but if one
        does occur it occurs on qubit #2 with a 72% chance.
    """

    # QHACK #
    probs = abs(np.array([circuit_output[i][i] for i in range(0, len(circuit_output))]))
    p_, p0, p1, p2 = 0, 0, 0, 0
    p_ = probs[0] + probs[4]
    if probs[7] != 0:
        p0 = probs[7] + probs[3]
    if probs[5] != 0:
        p1 = probs[5] + probs[3]
    if probs[6] != 0:
        p2 = probs[6] + probs[3]

    # QHACK #
    return [p_, p0, p1, p2]

In [5]:
prob = np.random.rand()
alpha = np.random.rand()
idx = np.random.randint(0, 3)
inputs = np.array([prob, alpha, idx], dtype=float)
p, alpha, tampered_wire = inputs[0], inputs[1], int(inputs[2])

error_readout = np.zeros(4, dtype=float)
circuit_output = circuit(p, alpha, tampered_wire)
error_readout = error_wire(circuit_output)
print(f"Probability of error not happening -> Predicted: {np.round(error_readout[0]*100, 2)}% vs Expected: {np.round((1 - inputs[0])*100,2)}%")
for i in range(0, 3):
    if inputs[-1] == i:
        prob = np.round(inputs[0]*100, 2)
    else:
        prob = 0
    print(f"Probability of error happening on wire {i}: {np.round(error_readout[i+1]*100, 2)}% vs Expected: {prob}%")

Probability of error not happening -> Predicted: 98.03% vs Expected: 98.03%
Probability of error happening on wire 0: 0% vs Expected: 0%
Probability of error happening on wire 1: 1.97% vs Expected: 1.97%
Probability of error happening on wire 2: 0% vs Expected: 0%


In [7]:
prob = np.random.rand()
alpha = np.random.rand()
for idx in [0, 1, 2]:
    inputs = np.array([prob, alpha, idx], dtype=float)
    p, alpha, tampered_wire = inputs[0], inputs[1], int(inputs[2])
    circuit_output = circuit(p, alpha, tampered_wire)
    diag = np.array([np.round(circuit_output[i][i].real, 4) for i in range(0, len(circuit_output))])
    diag = diag.reshape(2, 2, 2)
    print(f"Tampering wire {idx}")
    expected_output = [np.round((1 - inputs[0]), 4), 0, 0, 0]
    expected_output[idx+1] = np.round(inputs[0], 4)
    print(f"Expected output = {expected_output}")
    print(" Values for eachs tate")
    for i in [0, 1]:
        for j in [0, 1]:
            for k in [0, 1]:
                out = diag[i][j][k]
                print(f"  State|{i}{j}{k}> --> {out}")

Tampering wire 0
Expected output = [0.0722, 0.9278, 0, 0]
 Values for eachs tate
  State|000> --> 0.0288
  State|001> --> 0.0
  State|010> --> 0.0
  State|011> --> 0.3705
  State|100> --> 0.0434
  State|101> --> 0.0
  State|110> --> 0.0
  State|111> --> 0.5572
Tampering wire 1
Expected output = [0.0722, 0, 0.9278, 0]
 Values for eachs tate
  State|000> --> 0.0288
  State|001> --> 0.0
  State|010> --> 0.0
  State|011> --> 0.5572
  State|100> --> 0.0434
  State|101> --> 0.3705
  State|110> --> 0.0
  State|111> --> 0.0
Tampering wire 2
Expected output = [0.0722, 0, 0, 0.9278]
 Values for eachs tate
  State|000> --> 0.0288
  State|001> --> 0.0
  State|010> --> 0.0
  State|011> --> 0.5572
  State|100> --> 0.0434
  State|101> --> 0.0
  State|110> --> 0.3705
  State|111> --> 0.0
